In [1]:
import cv2
import os
from multiprocessing import Pool
import subprocess
from tqdm import tqdm
from PIL import Image, ImageDraw
import Matthieus_Functions as MF

import numpy as np


In [2]:
InputFile = '/mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/temp-09172024151650-crop.avi'
InFile2 ='/mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/MAX_20240917_151717_197__Channel488,525.avi'

In [44]:
cap = cv2.VideoCapture(InputFile)

In [46]:
resize = False
red = False

In [47]:
# Get total number of frames
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
new_width, new_height = 550, 650

# partition into less than 30MB videos

In [48]:
# Number of segments
num_parts = 20
frames_per_part = total_frames // num_parts

# Define codec and create VideoWriter objects
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # For AVI files
#fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Uncomment for MP4 files

output_files = [f"Behavior_{i+1}.avi" for i in range(num_parts)]
if resize:
    writers = [cv2.VideoWriter(output_files[i], fourcc, fps, (new_width, new_height)) for i in range(num_parts)]
else:
    writers = [cv2.VideoWriter(output_files[i], fourcc, fps, (frame_width, frame_height)) for i in range(num_parts)]

# Read and write frames to corresponding segments
current_part = 0
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if resize:
        resized_frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_AREA)
        red_channel = resized_frame.copy()
        if red:
            red_channel = resized_frame.copy()
            red_channel[:, :, 0] = 0  # Set blue channel to 0
            red_channel[:, :, 1] = 0  # Set green channel to 0
        
    if frame_idx // frames_per_part >= num_parts:
        break  # Stop if we exceeded the expected number of parts
    
    if resize:
        writers[current_part].write(red_channel)
    else:    
        writers[current_part].write(frame)

    frame_idx += 1
    if frame_idx % frames_per_part == 0 and current_part < num_parts - 1:
        current_part += 1  # Move to the next part

# Release everything
cap.release()
for writer in writers:
    writer.release()

print("Video split into 10 parts successfully.")

Video split into 10 parts successfully.


In [24]:
def save_frame(video_path, frame_num, result_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video was successfully opened
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    # Set the video position to the desired frame number
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

    # Read the frame
    ret, frame = cap.read()

    # If the frame was read successfully, save it
    if ret:
        cv2.imwrite(result_path, frame)
        print(f"Frame {frame_num} saved successfully at {result_path}")
    else:
        print(f"Error: Could not read frame {frame_num}")

    # Release the video capture object
    cap.release()

In [39]:
# Example usage
video_path = '/mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8.avi'
frame_num = 110  # Frame number to extract
path = video_path.split('.')[0]

result_path = path +'_fr' + str(frame_num) + '.jpg'
print(result_path)
save_frame(video_path, frame_num, result_path)

/mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr110.jpg
Frame 110 saved successfully at /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr110.jpg


In [ ]:
# Add a desired frame to the beginning of the movie:

In [40]:


def Add_fr0(image_path,video_path0,output_video_path):
    # Load the image to be added as the first frame
    image = cv2.imread(image_path)
    path = video_path0.split('.')[0]
    #output_video_path = path + '_fr0Added.avi'
    # Open the original video
    cap = cv2.VideoCapture(video_path0)
    
    # Check if the video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path0}")
        exit()
    
    # Retrieve original video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    
    
    # Create a VideoWriter object to write the new video
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    # Write the resized image as the first frame
    out.write(image)
    
    # Append the original video frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
    
    # Release resources
    cap.release()
    out.release()
    
    print(f"New video saved as {output_video_path}")

In [41]:
path = video_path.split('.')[0]
output_video_path = path + '_fr0Added.avi'
print(output_video_path)

/mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi


In [43]:
for frame_num in range(101,111):

    path = video_path.split('.')[0]
    
    result_path = path +'_fr' + str(frame_num) + '.jpg'
    Add_fr0(result_path,video_path)

[mpeg4 @ 0x1be5f80] Error at MB: 761
[mpeg4 @ 0x354f200] ac-tex damaged at 47 3
[mpeg4 @ 0x354f200] Error at MB: 221
[mpeg4 @ 0x1bb7200] ac-tex damaged at 12 15
[mpeg4 @ 0x1bb7200] Error at MB: 882


New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi
New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi
New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi


[mpeg4 @ 0x1bb6600] mcbpc damaged at 32 8
[mpeg4 @ 0x1bb6600] Error at MB: 496
[mpeg4 @ 0x1bb6600] illegal dc vlc
[mpeg4 @ 0x1bb6600] Error at MB: 1357
[mpeg4 @ 0x1baa200] ac-tex damaged at 45 7
[mpeg4 @ 0x1baa200] Error at MB: 451


New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi
New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi
New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi


[mpeg4 @ 0x6843700] ac-tex damaged at 36 6
[mpeg4 @ 0x6843700] Error at MB: 384
[mpeg4 @ 0x2b28700] ac-tex damaged at 43 13
[mpeg4 @ 0x2b28700] Error at MB: 797
[mpeg4 @ 0x2d262c0] 1. marker bit missing in 3. esc
[mpeg4 @ 0x2d262c0] Error at MB: 428


New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi
New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi
New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi
New video saved as /mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/output_part_8_fr0Added.avi


[mpeg4 @ 0x1bec0c0] Error at MB: 653


In [3]:
def convert_avi_to_mp4(directory):
    if not os.path.isdir(directory):
        print(f"The directory {directory} does not exist.")
        return

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if the file has an .avi extension
        if filename.lower().endswith('.avi'):
            avi_path = os.path.join(directory, filename)
            mp4_path = os.path.join(directory, f"{os.path.splitext(filename)[0]}.mp4")

            # Construct the ffmpeg command
            command = [
                'ffmpeg',
                '-i', avi_path,        # Input file
                '-c:v', 'libx264',     # Video codec
                '-c:a', 'aac',         # Audio codec
                '-b:a', '192k',        # Audio bitrate
                '-strict', 'experimental',
                mp4_path               # Output file
            ]

            # Execute the command
            try:
                subprocess.run(command, check=True)
                print(f"Successfully converted {filename} to MP4.")
            except subprocess.CalledProcessError as e:
                print(f"Error converting {filename}: {e}")

In [4]:
WorkDir = '/mnt/DATA/Mahsa/movies/LongRecordings/2024-09-17/W2tq9262/UpperCam_Devided/20pieces'
convert_avi_to_mp4(WorkDir)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 12.3.0 (conda-forge gcc 12.3.0-7)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1716729489913/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libo

Successfully converted Behavior_14.avi to MP4.


[out#0/mp4 @ 0x588b3fb93f00] video:10171KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.081228%
frame=  631 fps=213 q=-1.0 Lsize=   10179KiB time=00:02:37.25 bitrate= 530.3kbits/s speed=  53x    
[libx264 @ 0x588b3fbab040] frame I:14    Avg QP:14.34  size: 24762
[libx264 @ 0x588b3fbab040] frame P:177   Avg QP:17.02  size: 19823
[libx264 @ 0x588b3fbab040] frame B:440   Avg QP:17.87  size: 14907
[libx264 @ 0x588b3fbab040] consecutive B-frames:  6.0%  1.9%  3.3% 88.7%
[libx264 @ 0x588b3fbab040] mb I  I16..4:  2.3% 97.1%  0.5%
[libx264 @ 0x588b3fbab040] mb P  I16..4:  1.6% 54.2%  0.6%  P16..4: 21.2% 15.2%  7.1%  0.0%  0.0%    skip: 0.1%
[libx264 @ 0x588b3fbab040] mb B  I16..4:  0.7% 32.7%  0.1%  B16..8: 41.1% 15.1%  3.2%  direct: 5.6%  skip: 1.5%  L0:64.5% L1:19.3% BI:16.3%
[libx264 @ 0x588b3fbab040] 8x8 transform intra:96.9% inter:86.3%
[libx264 @ 0x588b3fbab040] coded y,uvDC,uvAC intra: 82.8% 85.7% 49.9% inter: 64.1% 67.2% 25.8%
[libx264 @ 0x588b3fb

Successfully converted Behavior_9.avi to MP4.


[out#0/mp4 @ 0x616cb629ca80] video:10402KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.079158%
frame=  631 fps=212 q=-1.0 Lsize=   10411KiB time=00:02:37.25 bitrate= 542.3kbits/s speed=52.9x    
[libx264 @ 0x616cb629ddc0] frame I:18    Avg QP:14.34  size: 25861
[libx264 @ 0x616cb629ddc0] frame P:186   Avg QP:17.11  size: 19844
[libx264 @ 0x616cb629ddc0] frame B:427   Avg QP:17.90  size: 15210
[libx264 @ 0x616cb629ddc0] consecutive B-frames:  7.1%  6.0%  5.7% 81.1%
[libx264 @ 0x616cb629ddc0] mb I  I16..4:  2.3% 97.2%  0.4%
[libx264 @ 0x616cb629ddc0] mb P  I16..4:  1.6% 56.9%  0.6%  P16..4: 20.2% 14.1%  6.5%  0.0%  0.0%    skip: 0.1%
[libx264 @ 0x616cb629ddc0] mb B  I16..4:  0.8% 33.4%  0.1%  B16..8: 40.7% 15.0%  3.2%  direct: 5.4%  skip: 1.3%  L0:65.0% L1:18.8% BI:16.2%
[libx264 @ 0x616cb629ddc0] 8x8 transform intra:96.9% inter:87.0%
[libx264 @ 0x616cb629ddc0] coded y,uvDC,uvAC intra: 83.1% 85.9% 50.9% inter: 65.2% 67.5% 26.5%
[libx264 @ 0x616cb62

...

Successfully converted Behavior_7.avi to MP4.


[out#0/mp4 @ 0x5bbe61b120c0] video:10918KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.074993%
frame=  631 fps=212 q=-1.0 Lsize=   10926KiB time=00:02:37.25 bitrate= 569.2kbits/s speed=52.8x    
[libx264 @ 0x5bbe61b28f40] frame I:21    Avg QP:14.56  size: 27645
[libx264 @ 0x5bbe61b28f40] frame P:188   Avg QP:17.26  size: 20942
[libx264 @ 0x5bbe61b28f40] frame B:422   Avg QP:18.10  size: 15785
[libx264 @ 0x5bbe61b28f40] consecutive B-frames:  8.4%  5.1%  6.7% 79.9%
[libx264 @ 0x5bbe61b28f40] mb I  I16..4:  2.0% 97.2%  0.8%
[libx264 @ 0x5bbe61b28f40] mb P  I16..4:  1.4% 56.8%  0.7%  P16..4: 20.2% 14.3%  6.6%  0.0%  0.0%    skip: 0.1%
[libx264 @ 0x5bbe61b28f40] mb B  I16..4:  0.7% 32.8%  0.2%  B16..8: 40.8% 15.0%  3.4%  direct: 5.8%  skip: 1.4%  L0:64.7% L1:17.4% BI:18.0%
[libx264 @ 0x5bbe61b28f40] 8x8 transform intra:97.0% inter:86.7%
[libx264 @ 0x5bbe61b28f40] coded y,uvDC,uvAC intra: 84.2% 86.7% 51.9% inter: 66.8% 68.2% 26.9%
[libx264 @ 0x5bbe61b

Successfully converted Behavior_5.avi to MP4.


Successfully converted Behavior_20.avi to MP4.


[out#0/mp4 @ 0x59934a1b8bc0] video:10097KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.080433%
frame=  631 fps=211 q=-1.0 Lsize=   10105KiB time=00:02:37.25 bitrate= 526.4kbits/s speed=52.5x    
[libx264 @ 0x59934a1ba980] frame I:22    Avg QP:14.37  size: 23252
[libx264 @ 0x59934a1ba980] frame P:187   Avg QP:17.14  size: 18950
[libx264 @ 0x59934a1ba980] frame B:422   Avg QP:17.84  size: 14889
[libx264 @ 0x59934a1ba980] consecutive B-frames:  8.9%  3.5%  7.1% 80.5%
[libx264 @ 0x59934a1ba980] mb I  I16..4:  2.3% 97.1%  0.6%
[libx264 @ 0x59934a1ba980] mb P  I16..4:  1.7% 56.5%  0.6%  P16..4: 21.3% 13.7%  6.1%  0.0%  0.0%    skip: 0.1%
[libx264 @ 0x59934a1ba980] mb B  I16..4:  0.8% 35.1%  0.2%  B16..8: 39.6% 14.7%  3.0%  direct: 5.2%  skip: 1.3%  L0:65.5% L1:19.4% BI:15.2%
[libx264 @ 0x59934a1ba980] 8x8 transform intra:96.9% inter:86.9%
[libx264 @ 0x59934a1ba980] coded y,uvDC,uvAC intra: 83.2% 84.8% 49.0% inter: 64.7% 66.0% 24.6%
[libx264 @ 0x59934a1